`comparaison_env`

In [1]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *

Variables globales

In [2]:
### MODELNET40 REMESHING ISO
ModelNet40_aligned_us = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R22.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print("Fichiers bvs de Modelnet40 :", len(paths_bvs))
dir_bvs = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/bvs_remeshing_iso"

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
dir_us = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Paths of the 44 csv file containing the results of the user study
paths_us_csv = glob.glob(os.path.join(dir_us, "**", "*tache_normalise.csv*"), recursive=True); print("Modeles de US : ",len(paths_us_csv))
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude.csv")
# Path data folder of user study
dir_Data = "/home/pelissier/These-ATER/Papier_internationale2/Data"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'aeroplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet', 'night_stand': 'cabinet', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}
# Outputs tmp
# Path of user-study outputs folder in Dataset
dir_outputs = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/user_study"


Fichiers bvs de Modelnet40 : 844
Modeles de US :  44


## Cameras

In [3]:
## User study
# Coordonnées des 8 caméras de l'étude utilateur
X_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['X_rep_etude'])
Y_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['Y_rep_etude'])
Z_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['Z_rep_etude'])

## ModeleNet 
# Coordonnées des 12 caméras de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])
cams_modelnet = np.column_stack((X_modelnet, Y_modelnet, Z_modelnet, np.array([1]*12)))

## TOUS les choix des utilisateurs --> couronne 'Milieu-Dessus' + ordre en fonction des choix

In [4]:
results = {}
# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_us_csv in paths_us_csv:
    df = pd.read_csv(path_us_csv)
    # name of model
    name = df['mesh_name'][0]
    # Choix des utilisateurs
    results[name] ={}
    results[name]['choix_1'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 1))]      
    results[name]['choix_2'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 2))]      
    results[name]['choix_3'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 3))]      

## Pour 1 categorie

In [5]:
categorie_modelnet = 'car'
categorie_us = match_ModelNet2US[categorie_modelnet]; print(categorie_modelnet, categorie_us)
## Fichiers BVS de la categorie
paths_bvs = [paths_bvs[i] for i in range(len(paths_bvs)) if categorie_modelnet in paths_bvs[i]]; print("Fichiers bvs de la categorie dispo :",len(paths_bvs), paths_bvs[0])

car carVasa
Fichiers bvs de la categorie dispo : 38 car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4


### Mise en correspondance cameras

In [6]:
## ModelNet Random
# Mesh random qui a ete aligne avec le mesh de l'étude utilisateur
path_mesh_bvs = paths_bvs[0]; print(path_mesh_bvs)
path_mesh_modelnet = os.path.join(ModelNet40_aligned_us, path_mesh_bvs+"_aligned_ok_US.obj"); print(path_mesh_modelnet)
mesh_modelnet = trimesh.load_mesh(path_mesh_modelnet)
# Transformation de la mesh de ModelNet40 en mesh de l'étude utilisateur
transformations = read_pkl(path_mesh_modelnet.replace(".obj", ".pkl"))
# Transformation des 12 cameras
cams_modelnet_mesh = cams_modelnet.copy()
for n in range(1, len(transformations)+1):
    # Rotation R
    R = transformations[f"transformations{n}"]
    cams_modelnet_mesh = np.dot(R, cams_modelnet_mesh.T).T
###################################################################
## User stuy
# mesh random from User_study
path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj"); print(path_mesh_us)
mesh_us = trimesh.load_mesh(path_mesh_us)

car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4
/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned/car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4_aligned_ok_US.obj
/home/pelissier/These-ATER/Papier_internationale2/Data/carVasa/carVasa_update_normed_centered_user_study.obj


In [16]:
colors_modelnet

array([[1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 0.64485259, 1.        ],
       [1.        , 1.        , 0.28970517, 1.        ],
       [1.        , 0.95637195, 0.        , 1.        ],
       [1.        , 0.70931353, 0.        , 1.        ],
       [1.        , 0.47254921, 0.        , 1.        ],
       [1.        , 0.23578489, 0.        , 1.        ],
       [0.9990205 , 0.        , 0.        , 1.        ],
       [0.75194424, 0.        , 0.        , 1.        ],
       [0.51516283, 0.        , 0.        , 1.        ],
       [0.27838141, 0.        , 0.        , 1.        ],
       [0.0416    , 0.        , 0.        , 1.        ]])

In [ ]:
## OBJ file with (12 cameras + obj)
# Vertices and faces of the model user study
verts_mesh_modelnet = np.array(mesh_modelnet.vertices)
faces_mesh_modelnet = np.array(mesh_modelnet.faces) 
# colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
colormap = plt.get_cmap('hot'); colors_modelnet = colormap(np.linspace(0, 1, 12))[::-1] 
# Write obj

with open(os.path.join(dir_outputs, categorie_modelnet+"_modelNet40+12cam.obj"), 'w') as obj_file:
    # Write vertices
    for vertex in verts_mesh_modelnet:
        obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
    
    # Add cubes
    for cube_center, cube_color in zip(cube_centers=cams_modelnet_mesh[:, :3], cube_colors=colors_modelnet[:,:3]):  
        vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset = len(verts_mesh_modelnet))
    
    # Write faces
    for face in faces_mesh_modelnet:
        # Convert face indices to 1-based indexing
        obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
 
## OBJ file with (8 cameras + obj)
# Vertices and faces of the model user study
verts_mesh_us = np.array(mesh_us.vertices)
faces_mesh_us = np.array(mesh_us.faces) 
# colors : Jaune cam 1 --> Vert cam 3 ---> Bleu cam 5 --> Bleu foncé cam 8
colormap = plt.get_cmap('viridis'); colors_us = colormap(np.linspace(0, 1, 8))[::-1] 
# Write obj
with open(os.path.join(dir_outputs, categorie_us+"_us+8cam.obj"), 'w') as obj_file:
    # Write vertices
    for vertex in verts_mesh_us:
        obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
    # Write camera positions
    for i in range(8):
        r, g, b, _ = colors_us[i]; print(r, g, b)
        obj_file.write(f"v {X_us[i]} {Y_us[i]} {Z_us[i]} {r*255} {g*255} {b*255}\n")
    # Write faces
    for face in faces_mesh_us:
        # Convert face indices to 1-based indexing
        obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
        """

'\nwith open(os.path.join(dir_outputs, categorie_modelnet+"_modelNet40+12cam.obj"), \'w\') as obj_file:\n    # Write vertices\n    for vertex in verts_mesh_modelnet:\n        obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      \n    \n    # Add cubes\n    # for cube_center, cube_color in zip(cube_centers=cams_modelnet_mesh[:, :3], cube_colors):  \n    #     vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset = len(verts_mesh_modelnet))\n    \n    # Write faces\n    for face in faces_mesh_modelnet:\n        # Convert face indices to 1-based indexing\n        obj_file.write(f"f {\' \'.join(str(idx + 1) for idx in face)}\n")   \n \n## OBJ file with (8 cameras + obj)\n# Vertices and faces of the model user study\nverts_mesh_us = np.array(mesh_us.vertices)\nfaces_mesh_us = np.array(mesh_us.faces) \n# colors : Jaune cam 1 --> Vert cam 3 ---> Bleu cam 5 --> Bleu foncé cam 8\ncolormap = plt.get_cmap(\'viridis\'); colors_us = colormap(np.lins

### BVS : match celle de l'US et celle de ModelNet40